In [1]:
import sqlite3
import ipycytoscape
import networkx as nx
from IPython.core.display import display, HTML

In [2]:
conn = sqlite3.connect("57149d10-831b-4c71-8f51-72497c4e1a50.sqlite")

In [3]:
print ("Opened database successfully")


Opened database successfully


In [4]:
mac_dict = {}
cursor_dst = conn.execute("SELECT DISTINCT ether_dhost from eth")
cursor_src = conn.execute("SELECT DISTINCT ether_shost from eth")
for row in cursor_dst:
    mac_dict[row[0]] = 1
for row in cursor_src:
    mac_dict[row[0]] = 1
    
macs = list(mac_dict.keys())
mac_nodes = []
for mac in macs:
    mac_nodes.append({"data": {"id": mac, "name": mac}})
cursor_edges = conn.execute("SELECT DISTINCT ether_shost, ether_dhost from eth")
mac_edges = []
for edge in cursor_edges:
    mac_edges.append({"data": {"source": edge[0], "target": edge[1]}})
data_macs = {"nodes": mac_nodes, "edges": mac_edges}

In [5]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(data_macs)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'content': 'data(name)',
                            'text-valign': 'center',
                            'color': 'white',
                            'text-outline-width': 1,
                            'text-outline-color': 'green',
                            'background-color': 'green'
                        }
                        },
                        {
                        'selector': 'edge',
                        'css': {
                            'line-color': 'black',
                        }}
])
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…

In [6]:
ip_dict = {}
cursor_dst = conn.execute("SELECT DISTINCT ip_dst from ip4")
cursor_src = conn.execute("SELECT DISTINCT ip_src from ip4")
for row in cursor_dst:
    ip_dict[row[0]] = 1
for row in cursor_src:
    ip_dict[row[0]] = 1
    
ips = list(ip_dict.keys())

ip_nodes = []
for ip in ips:
    ip_nodes.append({"data": {"id": ip, "name": ip}})
cursor_edges = conn.execute("SELECT DISTINCT ip_src, ip_dst from ip4")
ip_edges = []
for edge in cursor_edges:
    ip_edges.append({"data": {"source": edge[0], "target": edge[1]}})
data_ips = {"nodes": ip_nodes, "edges": ip_edges}

In [7]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(data_ips)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'content': 'data(name)',
                            'text-valign': 'center',
                            'color': 'white',
                            'text-outline-width': 1,
                            'text-outline-color': 'green',
                            'background-color': 'green'
                        }
                        },
                        {
                        'selector': 'edge',
                        'css': {
                            'line-color': 'black',
                        }}
])
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…

In [8]:
eth_dict = {}
cursor_eth = conn.execute("SELECT DISTINCT hash, ether_shost, ether_dhost from eth")
for row in cursor_eth:
    eth_dict[row[0]] = [row[1], row[2]]
    
dns_dict = {}
cursor_dns = conn.execute("SELECT DISTINCT ethh_hash, qname from dns")
for row in cursor_dns:
    dns_dict[row[0]] = row[1]

port_dict = {}
cursor_tcp = conn.execute("SELECT DISTINCT ethh_hash, source, dest from tcp")
for row in cursor_tcp:
    port_set = set()
    if row[0] in port_dict:
        port_set = port_dict[row[0]]
    
    if (row[1] < 10000):
        port_set.add(row[1])
    if (row[2] < 10000):
        port_set.add(row[2])
    port_dict[row[0]] = port_set

cursor_udp = conn.execute("SELECT DISTINCT ethh_hash, source, dest from udp")
for row in cursor_udp:
    port_set = set()
    if row[0] in port_dict:
        port_set = port_dict[row[0]]
    
    if (row[1] < 10000):
        port_set.add(row[1])
    if (row[2] < 10000):
        port_set.add(row[2])
    
    port_dict[row[0]] = port_set

In [14]:
ip4_dict = {}
ip4_port_dict = {}
cursor_ip4 = conn.execute("SELECT DISTINCT ethh_hash, ip_src, ip_dst from ip4")
for row in cursor_ip4:
    if row[0] in port_dict:
        ip4_port_dict[row[0]] = [row[1], row[2], list(port_dict[row[0]])]

    if row[0] in dns_dict and ("10.0.0." in row[1] or "10.0.0." in row[2]) and row[0] in eth_dict and not ("10.0.0.1" in row[1]):
        mac_src = eth_dict[row[0]][0]
        mac_dst = eth_dict[row[0]][1]
        if "10.0.0." in row[1]:
            ip4_dict[row[1]+dns_dict[row[0]]] = [mac_src, mac_dst, row[1], dns_dict[row[0]]]
        elif "10.0.0." in row[2]:
            ip4_dict[row[2]+dns_dict[row[0]]] = [mac_src, mac_dst, row[2], dns_dict[row[0]]]
address_nodes = []
for address in ip4_dict.keys():
    mac_addr = ip4_dict[address][0]+"/"+ip4_dict[address][1]
    address_nodes.append({"data": {"id": mac_addr, "name": mac_addr}})
    address_nodes.append({"data": {"id": ip4_dict[address][2], "name": ip4_dict[address][2]}})
    address_nodes.append({"data": {"id": ip4_dict[address][3], "name": ip4_dict[address][3]}})
    
address_edges = []
for address in ip4_dict.keys():
    mac_addr = ip4_dict[address][0]+"/"+ip4_dict[address][1]
    address_edges.append({"data": {"source": mac_addr, "target": ip4_dict[address][2]}})
    address_edges.append({"data": {"source": ip4_dict[address][2], "target": ip4_dict[address][3]}})

address_data = {"nodes": address_nodes, "edges": address_edges}

ip4_port_nodes = []
for hs in ip4_port_dict.keys():
    if (len(ip4_port_dict[hs][2]) > 0):
        ip4_port_nodes.append({"data": {"id": ip4_port_dict[hs][0], "name": ip4_port_dict[hs][0]}})
        ip4_port_nodes.append({"data": {"id": ip4_port_dict[hs][1], "name": ip4_port_dict[hs][1]}})
        for port in ip4_port_dict[hs][2]:
            ip4_port_nodes.append({"data": {"id": port, "name": port}})

ip4_port_edges = []
for hs in ip4_port_dict.keys():
    for port in ip4_port_dict[hs][2]:
        ip4_port_edges.append({"data": {"source": ip4_port_dict[hs][0], "target": port}})
        ip4_port_edges.append({"data": {"source": ip4_port_dict[hs][1], "target": port}})

ip4_port_data = {"nodes": ip4_port_nodes, "edges": ip4_port_edges}

In [10]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(address_data, directed=True)
cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e',
                            'content': 'data(name)',
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLengthVal': 10}, cytoscape_style=[{…

In [15]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(ip4_port_data, directed=True)
cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e',
                            'content': 'data(name)',
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLengthVal': 10}, cytoscape_style=[{…

In [18]:
ip4_val = "10.0.0.191"
ip4_port_nodes = []
for hs in ip4_port_dict.keys():
    if (len(ip4_port_dict[hs][2]) > 0) and (ip4_port_dict[hs][0] == ip4_val or ip4_port_dict[hs][1] == ip4_val):
        if (ip4_port_dict[hs][0] == ip4_val):
            ip4_port_nodes.append({"data": {"id": ip4_port_dict[hs][0], "name": ip4_port_dict[hs][0]}})
        elif (ip4_port_dict[hs][1] == ip4_val):
            ip4_port_nodes.append({"data": {"id": ip4_port_dict[hs][1], "name": ip4_port_dict[hs][1]}})
        for port in ip4_port_dict[hs][2]:
            ip4_port_nodes.append({"data": {"id": port, "name": port}})

ip4_port_edges = []
for hs in ip4_port_dict.keys():
    if (len(ip4_port_dict[hs][2]) > 0) and (ip4_port_dict[hs][0] == ip4_val or ip4_port_dict[hs][1] == ip4_val):
        if (ip4_port_dict[hs][0] == ip4_val):
            for port in ip4_port_dict[hs][2]:
                ip4_port_edges.append({"data": {"source": ip4_port_dict[hs][0], "target": port}})
        elif (ip4_port_dict[hs][1] == ip4_val):
            for port in ip4_port_dict[hs][2]:
                ip4_port_edges.append({"data": {"source": ip4_port_dict[hs][1], "target": port}})
ip4_port_data_filtered = {"nodes": ip4_port_nodes, "edges": ip4_port_edges}
print(ip4_port_data_filtered)

{'nodes': [{'data': {'id': '10.0.0.191', 'name': '10.0.0.191'}}, {'data': {'id': 67, 'name': 67}}, {'data': {'id': 68, 'name': 68}}, {'data': {'id': '10.0.0.191', 'name': '10.0.0.191'}}, {'data': {'id': 3591, 'name': 3591}}, {'data': {'id': 3602, 'name': 3602}}, {'data': {'id': 8213, 'name': 8213}}, {'data': {'id': 8728, 'name': 8728}}, {'data': {'id': 8734, 'name': 8734}}, {'data': {'id': 5172, 'name': 5172}}, {'data': {'id': 53, 'name': 53}}, {'data': {'id': 67, 'name': 67}}, {'data': {'id': 68, 'name': 68}}, {'data': {'id': 7244, 'name': 7244}}, {'data': {'id': 80, 'name': 80}}, {'data': {'id': 2643, 'name': 2643}}, {'data': {'id': 1624, 'name': 1624}}, {'data': {'id': 8283, 'name': 8283}}, {'data': {'id': 1628, 'name': 1628}}, {'data': {'id': 5222, 'name': 5222}}, {'data': {'id': 5228, 'name': 5228}}, {'data': {'id': 6253, 'name': 6253}}, {'data': {'id': 4720, 'name': 4720}}, {'data': {'id': 4213, 'name': 4213}}, {'data': {'id': 8312, 'name': 8312}}, {'data': {'id': 6265, 'name': 6

In [19]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(ip4_port_data_filtered, directed=True)
cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e',
                            'content': 'data(name)',
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLengthVal': 10}, cytoscape_style=[{…